ME21B174 ASSIGNMENT 6

Importing the necessaary libraries

In [1]:
import numpy as np
import pandas as pd

# Task 1

Note : In the dataset I downloaded, there was no colummn named "token". So, For now I have taken other 3 columns as required

Set of languages required

In [2]:
languages = ["af-ZA", "cy-GB", "da-DK", "de-DE", "en-US", "es-ES", "fr-FR", "fi-FI", "hu-HU", "is-IS", "it-IT", "jv-ID", "lv-LV", "ms-MY", "nb-NO", "nl-NL", "pl-PL", "pt-PT", 
"ro-RO", "ru-RU", "sl-SL", "sv-SE", "sq-AL", "sw-KE", "tl-PH", "tr-TR", "vi-VN"]

Before running the next part of the code, create a new folder named "task1" in the assignement folder. The following code will create a csv file for each language in the task1 folder and store the required data of each language in it.  

In [3]:
for lang in languages:
    
    data = pd.read_json(f"1.0/data/{lang}.jsonl", lines=True)
    df = data["utt"]
    df.to_csv(f"task1/{lang}.csv")

# Task 2

Creating a master dataframe which contains the required data from all 27 languages

In [4]:
columns_req =  ["locale", "partition", "utt",] ## COLUMNS OF INTEREST
master_df = pd.DataFrame()
for lang in languages:
    df = pd.read_json(f"1.0/data/{lang}.jsonl", lines=True)
    df = df[columns_req]
    master_df = pd.concat([master_df, df], axis=0) ## CONCAT

master_df.reset_index(inplace=True, drop=True) ## RESETTING INDEX

In [5]:
master_df

,locale,partition,utt
0,af-ZA,test,maak my wakker om vyf v. m. die week
1,af-ZA,train,maak my wakker nege-uur v. m. op vrydag
2,af-ZA,train,stel 'n alarm vir twee ure van nou af
3,af-ZA,test,stil
4,af-ZA,train,janneman stilte
...,...,...,...
446062,vi-VN,train,tôi có email nào không
446063,vi-VN,train,những thư điện tử nào là mới
446064,vi-VN,train,tôi có thư điện tử mới nào từ huy không
446065,vi-VN,test,hùng có gửi tôi email không


To use naive bayes classifier, we need to convert the features and label to numerical values. I have used label encoder for encoding label and countvectorizer to convert features

Importing necessary libraries

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder


In [7]:
encoder = LabelEncoder()
master_df["locale"] = encoder.fit_transform(master_df["locale"])

In [8]:
master_df["locale"].unique()

array([ 0,  1,  2,  3,  4,  5,  7,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 22, 21, 23, 24, 25, 26])

## Train - test - val split

In [9]:
X_train = master_df[master_df["partition"] == "train"]["utt"]
y_train = master_df[master_df["partition"] == "train"]["locale"]
X_test = master_df[master_df["partition"] == "test"]["utt"]
y_test = master_df[master_df["partition"] == "test"]["locale"]
X_dev = master_df[master_df["partition"] == "dev"]["utt"]
y_dev = master_df[master_df["partition"] == "dev"]["locale"]

CountVectorizer converts a collection of text documents into a matrix of token counts. It tokenizes the text, builds a vocabulary of unique words, and encodes each document as a sparse matrix where each entry represents the frequency of a word in that document.


In [10]:
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
X_dev = vectorizer.transform(X_dev)

## Finding the optimal alpha for the naive bayes classifier

In [11]:
alphas = np.linspace(0.1, 1, 10)
best_score = 0
best_alpha = 0
for alpha in alphas:
    classifier = MultinomialNB(alpha=alpha)
    classifier.fit(X_train, y_train)
    score = classifier.score(X_dev, y_dev)
    if score > best_score:
        best_score = score
        best_alpha = alpha
print(f"Best alpha : {best_alpha}, Best score : {best_score}")

Best alpha : 0.1, Best score : 0.9851159570785739


In [12]:
best_classifier = MultinomialNB(alpha=best_alpha)
best_classifier.fit(X_train, y_train)
print(f"Test score : {best_classifier.score(X_test, y_test)}") 

Test score : 0.9847567809908092


We can see that the model performs well for this dataset

# Task 3

Creating 4 language groups based on continent

In [13]:
# Asia
asia = ["jv-ID", "ms-MY", "tl-PH", "tr-TR", "vi-VN"]

# Africa
africa = ["af-ZA", "sw-KE"]

# Europe
europe = ["cy-GB", "da-DK", "de-DE", "es-ES", "fr-FR", "fi-FI", "hu-HU", "is-IS", "it-IT", 
          "lv-LV", "nb-NO", "nl-NL", "pl-PL", "pt-PT", "ro-RO", "ru-RU", "sl-SL", "sv-SE", "sq-AL"]

# North America
north_america = ["en-US"]

In [14]:
def findContinent(language): ## RETURN CONTINENT
    if language in asia:
        return "Asia"
    elif language in africa:
        return "Africa"
    elif language in europe:
        return "Europe"
    elif language in north_america:
        return "North America"
    else:
        return "Unknown"

Adding a seperate column named "continent" which denotes the continent of each datapoint in the dataset. 

In [15]:
columns_req =  ["locale", "partition", "utt",]
continental_df = pd.DataFrame()
for lang in languages:
    df = pd.read_json(f"1.0/data/{lang}.jsonl", lines=True)
    df = df[columns_req]
    df["Continent"] = findContinent(lang)
    continental_df = pd.concat([continental_df, df], axis=0)

continental_df.reset_index(inplace=True, drop=True)


In [16]:
continental_df

,locale,partition,utt,Continent
0,af-ZA,test,maak my wakker om vyf v. m. die week,Africa
1,af-ZA,train,maak my wakker nege-uur v. m. op vrydag,Africa
2,af-ZA,train,stel 'n alarm vir twee ure van nou af,Africa
3,af-ZA,test,stil,Africa
4,af-ZA,train,janneman stilte,Africa
...,...,...,...,...
446062,vi-VN,train,tôi có email nào không,Asia
446063,vi-VN,train,những thư điện tử nào là mới,Asia
446064,vi-VN,train,tôi có thư điện tử mới nào từ huy không,Asia
446065,vi-VN,test,hùng có gửi tôi email không,Asia


Seggregating the master dataset based on continents present 

In [17]:
african_langs = continental_df[continental_df["Continent"] == "Africa"].reset_index(inplace=False, drop=True)
asian_langs = continental_df[continental_df["Continent"] == "Asia"].reset_index(inplace=False, drop=True)
european_langs = continental_df[continental_df["Continent"] == "Europe"].reset_index(inplace=False, drop=True)
north_american_langs = continental_df[continental_df["Continent"] == "North America"].reset_index(inplace=False, drop=True)

Before running the next part of the code, create a new folder named "task3" in the assignement folder. The following code will create a csv file for each continent in the task3 folder and store the required data of each continent in it.  


In [18]:
african_langs.to_csv("task3/african.csv")
asian_langs.to_csv("task3/asian.csv")
european_langs.to_csv("task3/european.csv")
north_american_langs.to_csv("task3/north_american.csv")

Import the libraries for LDA and QDA

In [19]:
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA)

## TRAIN - TEST - VAL SPLIT

In [20]:
task3_train = pd.concat([african_langs[african_langs["partition"] == "train"], asian_langs[asian_langs["partition"] == "train"],
                         european_langs[european_langs["partition"] == "train"], north_american_langs[north_american_langs["partition"] == "train"]],
                         axis=0).reset_index(inplace=False, drop=True)

task3_test = pd.concat([african_langs[african_langs["partition"] == "test"], asian_langs[asian_langs["partition"] == "test"],
                         european_langs[european_langs["partition"] == "test"], north_american_langs[north_american_langs["partition"] == "test"]],
                         axis=0).reset_index(inplace=False, drop=True)

task3_dev = pd.concat([african_langs[african_langs["partition"] == "dev"], asian_langs[asian_langs["partition"] == "dev"],
                         european_langs[european_langs["partition"] == "dev"], north_american_langs[north_american_langs["partition"] == "dev"]],
                         axis=0).reset_index(inplace=False, drop=True)

In [21]:
X_train = task3_train["utt"]
y_train = task3_train["Continent"]

X_test = task3_test["utt"]
y_test = task3_test["Continent"]

X_dev = task3_dev["utt"]
y_dev = task3_dev["Continent"]

Using Tfidf vectorizer to create the feature space of the dataset

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
t_vectorizer = TfidfVectorizer()

X_train = t_vectorizer.fit_transform(X_train)
X_test = t_vectorizer.transform(X_test)
X_dev = t_vectorizer.transform(X_dev)

In [23]:
X_train

<310878x152098 sparse matrix of type '<class 'numpy.float64'>'
	with 1933920 stored elements in Compressed Sparse Row format>

In [24]:
X_train.shape

(310878, 152098)

## Feature Reduction

I have used SVD to do feature elimination to reduce from 152098 to 300 features.

In [25]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
X_train = svd.fit_transform(X_train)

In [26]:
X_train.shape

(310878, 300)

In [27]:
X_dev = svd.transform(X_dev)
X_test = svd.transform(X_test)

## LDA

In [28]:
lda = LDA(store_covariance=True)

lda.fit(X_train, y_train)

LinearDiscriminantAnalysis(store_covariance=True)

In [29]:
print("Dev score: ", lda.score(X_dev, y_dev))
print("Test score: ", lda.score(X_test, y_test))

Dev score:  0.9316281357599606
Test score:  0.9300480709357642


## QDA

In [30]:
qda = QDA(store_covariance=True)

qda.fit(X_train, y_train)

QuadraticDiscriminantAnalysis(store_covariance=True)

In [31]:
print("Dev score: ", qda.score(X_dev, y_dev))
print("Test score: ", qda.score(X_test, y_test))

Dev score:  0.9044287770308429
Test score:  0.9041196542877781


## Analysis to prepare the RDA Model

In [32]:
np.array(qda.covariance_).shape

(4, 300, 300)

In [33]:
np.array(lda.covariance_).shape

(300, 300)

All the classes have same covariance matrix (Assumption taken in LDA) whereas each class have different covariance matrix in QDA. Hence, there is diffference in shape of the covariance matrices

In [34]:
lda.means_.shape

(4, 300)

In [35]:
qda.means_.shape

(4, 300)

In [36]:
lda.priors_

array([0.07407407, 0.18518519, 0.7037037 , 0.03703704])

In [37]:
qda.priors_

array([0.07407407, 0.18518519, 0.7037037 , 0.03703704])

In [38]:
np.linalg.det(qda.covariance_)

array([0., 0., 0., 0.])

## OBSERVATIONS

We can see that both lda and qda has same means and priors. But the determinant of covariance matrix of dataset is 0 as we have used 300 features which is still a bit large.  So , in the following RDA, no. of features have been reduced to 15. This is optimal for RDA even though it might not be enough features for qda to work properly

## RDA

## TRAIN - TEST - VAL SPLIT

In [39]:
X_train = task3_train["utt"]
y_train = task3_train["Continent"]

X_test = task3_test["utt"]
y_test = task3_test["Continent"]

X_dev = task3_dev["utt"]
y_dev = task3_dev["Continent"]

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
t_vectorizer = TfidfVectorizer()

X_train = t_vectorizer.fit_transform(X_train)
X_test = t_vectorizer.transform(X_test)
X_dev = t_vectorizer.transform(X_dev)

### Feature Reduction

In [41]:
from sklearn.decomposition import TruncatedSVD

svd2 = TruncatedSVD(n_components=15, random_state=42) ## 15 features taken
X_train = svd2.fit_transform(X_train)
X_dev = svd2.transform(X_dev)
X_test = svd2.transform(X_test)

In [42]:
X_train.shape

(310878, 15)

CREATING A CLASS FOR RDA

In [43]:
class RDA:
    def __init__(self, lam=0.5) -> None:
        self.lam = lam

    def fit(self, X, y): ## FINDS THE OUTPUTS OF BOTH LDA AND QDA
        self.classes_ = np.unique(y)
        lda = LDA(store_covariance=True)
        qda = QDA(store_covariance=True)
        
        lda.fit(X, y)
        qda.fit(X, y)

        qda_cov = np.array(qda.covariance_)
        lda_cov = np.stack([np.array(lda.covariance_)] * qda_cov.shape[0])

        ##  lam is the tradeoff between LDA and QDA

        self.covariance_ = (1-self.lam) * np.array(qda_cov) + self.lam * np.array(lda_cov) 
        self.means_ = lda.means_           # Both LDA and QDA have same means 
        self.priors_ = lda.priors_          # Both LDA and QDA have same priors

    ## The function calculates the log of the likelihood for each class using the Gaussian formula and adds the log of the prior probabilities.
    ## It selects the class with the highest score as the predicted class for each sample.
    def predict(self, X):
        n, d = X.shape
        self.scores = np.zeros((n, len(self.priors_)))
        self.det_cov = np.linalg.det(self.covariance_)
        self.inv_cov = np.linalg.inv(self.covariance_)
    
        for idx, cl in enumerate(self.classes_):
            mean_vec = self.means_[idx]

            diff = X - mean_vec 
            score = -0.5 * np.sum(diff @ self.inv_cov[idx] * diff, axis=1)  
            score -= 0.5 * np.log(self.det_cov[idx])
            score += np.log(self.priors_[idx])
            self.scores[:, idx] = score

        return self.classes_[np.argmax(self.scores, axis=1)]
    
    def score(self, X, y):
        y_pred = self.predict(X_test)
        correct_predictions = np.sum(y_pred == y_test)

        accuracy = correct_predictions / len(y_test)
        return accuracy

## Finding optimal lambda for RDA

In [44]:
lambdas = np.linspace(0.0, 1.0, 11)
print(lambdas)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [45]:
best_score = 0
best_lam = 0
 
for lam in lambdas:
    rda = RDA(lam=lam)
    rda.fit(X_train, y_train)
    print(f"Lamda: {lam}, Score:{rda.score(X_test, y_test)}")

    if rda.score(X_test, y_test) > best_score:
        best_score = rda.score(X_test, y_test)
        best_lam = lam

print(f"Best Lambda: {best_lam}")

Lamda: 0.0, Score:0.5682831452838177
Lamda: 0.1, Score:0.6158061221948243
Lamda: 0.2, Score:0.8659244314926897
Lamda: 0.30000000000000004, Score:0.860581832673292
Lamda: 0.4, Score:0.8549278935963536
Lamda: 0.5, Score:0.8517024085282323
Lamda: 0.6000000000000001, Score:0.8488505317691599
Lamda: 0.7000000000000001, Score:0.8468205932899948
Lamda: 0.8, Score:0.8449525517447508
Lamda: 0.9, Score:0.8432713143540312
Lamda: 1.0, Score:0.8412040150439613
Best Lambda: 0.2


In [46]:
print(f"Best Score for RDA: {best_score}")

Best Score for RDA: 0.8659244314926897
